# 🧠 Crop Disease Classification with CNN (PlantVillage via TFDS)

This Colab notebook loads the PlantVillage dataset, trains a CNN model, and saves it for deployment.

## 📥 Step 1: Install Required Libraries (if needed)

In [ ]:
!pip install tensorflow tensorflow-datasets matplotlib

## 📦 Step 2: Import Libraries

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt


## 🌱 Step 3: Load the PlantVillage Dataset from TFDS

In [ ]:
(ds_train, ds_val), ds_info = tfds.load(
    'plant_village',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

class_names = ds_info.features['label'].names
num_classes = ds_info.features['label'].num_classes

print(f"Total Classes: {num_classes}")
print("Sample classes:", class_names[:5])


## 🧼 Step 4: Preprocess & Visualize

In [ ]:
def format_example(image, label):
    image = tf.image.resize(image, (128, 128)) / 255.0
    return image, label

ds_train = ds_train.map(format_example).batch(32).prefetch(tf.data.AUTOTUNE)
ds_val = ds_val.map(format_example).batch(32).prefetch(tf.data.AUTOTUNE)

# Show sample
for images, labels in ds_train.take(1):
    plt.imshow(images[0])
    plt.title(f"Label: {class_names[labels[0].numpy()]}")
    plt.axis('off')
    plt.show()


## 🧠 Step 5: Build CNN Model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


## 🏋️‍♀️ Step 6: Train the Model

In [ ]:
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=10
)


## 📈 Step 7: Evaluate

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


## 💾 Step 8: Save the Model

In [ ]:
model.save("crop_model.h5")